# DART Report Crawling

## 0. imports

In [1]:
%load_ext jupyter_black

In [2]:
import sys

sys.path.append("..")

In [3]:
import os
import re
import requests

from src.OpenDartReader import OpenDartReader
from bs4 import BeautifulSoup

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [5]:
dart = OpenDartReader(os.environ["DART_API_KEY"])

## 1. 공시검색 (모든종목)

- `start`와 `end`는 3개월 이내 범위만 가능함

In [6]:
# 2022-01-01 ~ 2022-01-10 모든 회사의 모든 공시목록 (3,139 건)

df = dart.list(start="2022-01-01", end="2022-01-03")

In [ ]:
len(df)

In [8]:
rcept_no_list = df["rcept_no"].tolist()

## 2. 공시문서 크롤링

In [ ]:
rcept_no = rcept_no_list[-2]

# xml_text = dart.document(rcept_no)
doc_df = dart.sub_docs(rcept_no)

In [10]:
# document = {}
# for idx, row in doc_df.iterrows():
#     url = row["url"]
#     title = row["title"]
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, "html.parser")
#     text = soup.get_text(strip=False)
#     text = re.sub(r"\n+", "\n", text)
#     text = re.sub(r" {2,}", " ", text)

#     document["title"] = title
#     document["text"] = text

In [10]:
import sys

sys.path.append("..")
from src.utils import ProxyUserAgentManager

PROXI_MANAGER = ProxyUserAgentManager()

In [19]:
import difflib
import pandas as pd
from fake_useragent import UserAgent

USER_AGENT = UserAgent()
USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.3904.108 Safari/537.36"

In [26]:
config = PROXI_MANAGER.get_next_proxy_user_agent()
proxies = {"http": "http://203.0.113.0:8443", "https": "https://203.0.113.0:8443"}
headers = {
    "User-Agent": config["user_agent"],
}

In [ ]:
proxies, headers

In [28]:
rcp_no = "20220816001711"
match = None

In [ ]:
if rcp_no.isdecimal():
    r = requests.get(
        f"http://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}",
        headers=headers,
        proxies=proxies,
    )
elif rcp_no.startswith("http"):
    r = requests.get(rcp_no, headers=headers, proxies=proxies)
else:
    raise ValueError("invalid `rcp_no`(or url)")

## 하위 문서 URL 추출
multi_page_re = (
    "\s+node[12]\['text'\][ =]+\"(.*?)\"\;"
    "\s+node[12]\['id'\][ =]+\"(\d+)\";"
    "\s+node[12]\['rcpNo'\][ =]+\"(\d+)\";"
    "\s+node[12]\['dcmNo'\][ =]+\"(\d+)\";"
    "\s+node[12]\['eleId'\][ =]+\"(\d+)\";"
    "\s+node[12]\['offset'\][ =]+\"(\d+)\";"
    "\s+node[12]\['length'\][ =]+\"(\d+)\";"
    "\s+node[12]\['dtd'\][ =]+\"(.*?)\";"
    "\s+node[12]\['tocNo'\][ =]+\"(\d+)\";"
)
matches = re.findall(multi_page_re, r.text)

In [20]:
if len(matches) > 0:
    row_list = []
    for m in matches:
        doc_id = m[1]
        doc_title = m[0]
        params = f"rcpNo={m[2]}&dcmNo={m[3]}&eleId={m[4]}&offset={m[5]}&length={m[6]}&dtd={m[7]}"
        doc_url = f"http://dart.fss.or.kr/report/viewer.do?{params}"
        row_list.append([doc_title, doc_url])
    df = pd.DataFrame(row_list, columns=["title", "url"])
    if match:
        df["similarity"] = df["title"].apply(
            lambda x: difflib.SequenceMatcher(None, x, match).ratio()
        )
        df = df.sort_values("similarity", ascending=False)
else:
    single_page_re = (
        "\t\tviewDoc\('(\d+)', '(\d+)', '(\d+)', '(\d+)', '(\d+)', '(\S+)',''\)\;"
    )
    matches = re.findall(single_page_re, r.text)
    if len(matches) > 0:
        doc_title = BeautifulSoup(r.text, features="lxml").title.text.strip()
        m = matches[0]
        params = f"rcpNo={m[0]}&dcmNo={m[1]}&eleId={m[2]}&offset={m[3]}&length={m[4]}&dtd={m[5]}"
        doc_url = f"http://dart.fss.or.kr/report/viewer.do?{params}"
        df = pd.DataFrame([[doc_title, doc_url]], columns=["title", "url"])

In [ ]:
df

In [ ]:
matches

In [ ]:
f"http://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}"